# Functions for calculating reachability set support function
Andrey Tremba, atremba@ipu.ru  
17.12.2021 (update)


```python
# 'ipynb as module' loader
import ipynb_loader
# import ipynb itself
import reachability_set_utils as reach_set

FIXME

```

see also examples:
 - [example-2d.ipynb](example-2d.ipynb) - 2D reachability sets and control

In [2]:
import numpy as np
import scipy.integrate # integration

In [3]:
def instant_universal_control_s(s, p, exp_func, sup_func_u, B=None):
    """
    Calculate optimal control hat(u)(s) element for reaching reachability set M(T).
    Real control shall be u(t) = hat(u)(T - t), t in [0, T]
    
    s - time instant
    p - vector (direction)
    exp_func - function calculating exp(A s) with signature E = exp_func(s)
    sup_func_u - function calculating support element a_U(p) with signature
        a = sup_func_u(p)
    
    17.01.2022 - public release
    Andrey Tremba
    """
    e_As = exp_func(s)
    if B is None:
        B = np.eye(len(p))
    q = B.T @ e_As.T @ p
    sup_element_u = sup_func_u(q)
    
    return sup_element_u

def instant_universal_sup_element(s, p, exp_func, sup_func_u):
    """
    Calculate support element for reachability set M given 
        matrix exponential e^{A s} B and support function / element of control set U

    s - time instant
    p - vector (direction)
    exp_func - function calculating exp(A s) with signature E = exp_func(s)
    sup_func_u - function calculating support element a_U(p) with signature
        a = sup_func_u(p)

    17.01.2022 - public release
    Andrey Tremba
    """
    e_AsB = exp_func(s)
    q = e_AsB.T @ p
    sup_element_u = sup_func_u(q)
    sup_element_m = e_AsB.dot(sup_element_u) # operator @ fails for vector e_AsB and scalar u 

    sup_value_num = sup_element_m @ p
    
    return sup_element_m

def get_reachability_set_support(p, T, exp_func, sup_func_u, B=None, 
                                 options={'quad_vec':{}, 'int_type':'vector'}):
    """
    Calculate support element of reachability set to instant T
    
    p - vector (direction)
    T - final time
    exp_func - function calculating exp(A s) with signature E = exp_func(s)
    sup_func_u - function calculating support element a_U(p) with signature
        a = sup_func_u(p)
    B - (optionally) input-forming matrix in x' = A x + B u, by default identity 
        matrix
    options - dictionary with data, containing at keys:
        'quad_vec' - dictionary with options for vector integration utility quad_vec
        'int_type' - RESEREVED. string with integration type. Possible values are:
            'vector' - use vector intergation utility
            'componentwise' - integrate each component separately
            
    Returns a support element

    17.01.2022 - public release
    Andrey Tremba
    
    TODO: check options dict to contain quad_vec options.
    """
    if B is not None: # empty B means there is no B matrix in system
        # setting it to eye(n) will harm performance (and its size is unknown, btw)
        local_exp_func = lambda s : exp_func(s).dot(B)
    else:
        local_exp_func = exp_func

    # try to evaluate support element at one instant
    tmp_sup_element = instant_universal_sup_element(T, p, local_exp_func, sup_func_u)
    
    int_func = lambda s : instant_universal_sup_element(s, p, local_exp_func, sup_func_u)
    
    if options['int_type'] == 'componentwise':
        integral = np.array([scipy.integrate.quad(
            lambda s : int_func(s)[k], 0, T, epsabs=1e-20, epsrel=1e-10, limit=2000)[0] # , epsabs=1e-14, epsrel=1e-18, limit=2000
                for k in range(len(tmp_sup_element))])
    elif options['int_type'] == 'vector':
        integral, quality = scipy.integrate.quad_vec(int_func, 0, T, epsabs=1e-16, epsrel=1e-10, norm='max', 
                                                    quadrature='gk15')
    else:
        raise('Unknown "int_type" option.');

    return integral

In [3]:
# import numpy as np

# T = 2
# sigma = 0.5

# def expm3_num(t, sigma=1):
#     """Matrix exponential exp(A t) of Jordan block with given eigenvalue -sigma"""
#     return np.exp(-sigma * t) * np.array([[1, t, t**2/2], [0, 1, t], [0, 0, 1]])

# exp_func_A = lambda t : expm3_num(t, sigma)
# sup_func_u = lambda q : 10 * q / np.sqrt(np.sum(q**2))
# reachability_support_u_ball = lambda p, T : \
#     get_reachability_set_support(p, T, exp_func_A, sup_func_u)

# p = np.random.randn(3)
# print(f'p = {p}')
# exp_func_u_ball = reachability_support_u_ball(p, T)

# print(exp_func_u_ball)

In [4]:
# import time

# # utitilites in ipynb files
# import ipynb_loader # loader
# # import image_utils # equal 3d axis
# import quasi_uniform_3d as qu3d # sphere point sampling
# # import polynomial_utils as PolyU # analysis and integration of polynomials
# import convex_utils # maximum radius, etc

In [5]:
# N = 20 # N = 60
# T = 2

# dir_points = qu3d.generate_sphere_uniform_grid(N) # quasi-uniform grid
# print(f'Grid has {len(dir_points)} points')


# t_begin = time.perf_counter()
# supp_points_ball = np.array([reachability_support_u_ball(p, T) for p in dir_points])
# t_end = time.perf_counter()

# print(f'Integration of universal function for ball: {t_end - t_begin} seconds elapsed')

In [6]:
# RR_ball = convex_utils.calculate_radiuses(dir_points, supp_points_ball)
# print(f'Bounding ball radius and its lower bound for ball control : {RR_ball[:2]},\n'
#       f'  difference = {RR_ball[0] - RR_ball[1]}')

In [7]:
# # exp_func_A_segment = lambda t : expm3_num(t, sigma)
# sup_func_u_segment = lambda q : np.sign(q)

# B = np.array([0, 0, 1])
# reachability_support_u_segment_componentwise = lambda p, T : \
#     get_reachability_set_support(p, T,
#                                  exp_func_A,
#                                  sup_func_u_segment, 
#                                  B, options={'int_type' : 'componentwise'})

# reachability_support_u_segment_vector = lambda p, T : \
#     get_reachability_set_support(p, T,
#                                  exp_func_A,
#                                  sup_func_u_segment,
#                                  B, options={'int_type' : 'vector'})


In [8]:
# t_begin = time.perf_counter()
# supp_points_segment_componentwise = np.array([reachability_support_u_segment_componentwise(p, T) for p in dir_points])
# t_end = time.perf_counter()

# print(f'Segment control, componentwise integration: {t_end - t_begin} seconds elapsed');

In [9]:
# t_begin = time.perf_counter()
# supp_points_segment = np.array([reachability_support_u_segment_vector(p, T) for p in dir_points])
# t_end = time.perf_counter()

# print(f'Segment control, vector integration: {t_end - t_begin} seconds elapsed');

# # np.testing.assert_allclose(supp_points_segment_componentwise, supp_points_segment)
# print(f'Max abs difference between componentwise and vector integration is '
#       f'{np.max(supp_points_segment_componentwise - supp_points_segment)}')

In [10]:
# RR_segment = calculate_radiuses(dir_points, supp_points_segment)

# print(f'Bounding ball radius and its lower bound for segment control : {RR_segment[:2]},\n'
#       f'  difference = {RR_segment[0] - RR_segment[1]}')

In [11]:
# supp_points_joint = supp_points_ball + supp_points_segment

# RR_joint = calculate_radiuses(dir_points, supp_points_joint)

# print(f'Bounding ball radius and its lower bound for joint control (segment + ball) : {RR_joint[:2]},\n'
#       f'  difference = {RR_joint[0] - RR_joint[1]}')

# B = np.array([0, 0, 1])
# sup_func_Bu_segment = lambda q_full : B * sup_func_u_segment(B.T @ q_full)

In [12]:
# reachability_support_u_joint = lambda p, T : \
#     get_reachability_set_support(p, T,
#                                  exp_func_A, 
#                                  lambda q : sup_func_Bu_segment(q) + sup_func_u(q), 
#                                  options={'int_type' : 'vector'})

In [13]:
# t_begin = time.perf_counter()
# supp_points_joint_2 = np.array([reachability_support_u_joint(p, T) for p in dir_points])
# t_end = time.perf_counter()

# print(f'Joint control, calculated directly, vector integration: {t_end - t_begin} seconds elapsed');

# print(f'Max abs difference between sum of support points and explicit calculation is '
#       f'{np.max(supp_points_joint - supp_points_joint_2)}')

In [14]:
# RR_joint_2 = calculate_radiuses(dir_points, supp_points_joint_2)

# print(f'{np.abs(RR_joint[0] - RR_joint_2[0])}, {np.abs(RR_joint[1] - RR_joint_2[1])}')
